In [1]:
import numpy as np
import librosa # conda install -c conda-forge librosa
import librosa.display

import os  # for mkdir
from os import listdir
from os.path import isfile, join
import sys

%matplotlib inline
import matplotlib.pyplot as plt
import math

from scipy import signal
import soundfile as sf

##-----------   paramManager  interface   ------------------##
# git clone https://github.com/prashanthtr/paramManager
from paramManager import paramManager

In [2]:
datapath='./'  #root folder
fileList = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f)) and (os.path.splitext(f)[1]==".wav" or os.path.splitext(f)[1]==".aif" or os.path.splitext(f)[1]==".aiff")]
#makeDirs = [os.makedirs(os.path.splitext(f)[0]) for f in fileList if not os.path.exists(os.path.splitext(f)[0])] #creates Directories

### Chunk file into 1 seconds and based on SampleRate

In [3]:
#fileList = ["silence.wav"]
createSegments = True

# returns the longest substring less than cutoff value
# Used to determine whether array is silent
def lsubstr(arr,e):
    substrLen = []
    cnt = 0
    for x in arr:
        if abs(x) < e:
            cnt = cnt+1;
        else:
            substrLen.append(cnt)
            cnt=0;
    if len(substrLen)==0:
            substrLen.append(cnt)
    print("max",max(substrLen))
    return max(substrLen)<=len(arr)/2


if createSegments == True:
    for fileName in fileList:

        sig,sr=librosa.core.load(fileName, sr=None)
        Fs = 22050

        # resample
        sig= librosa.core.resample(sig, sr, 22050) 

        fileBase = os.path.splitext(fileName)[0]

        chunkDuration = 1; # 1 sec
        numChunks = math.floor(len(sig)/Fs)
        numSamplesPerChunk = chunkDuration*Fs;
        segment = []

        mean = np.mean(sig)

        for i in range(numChunks-1):
            segment = sig[i*numSamplesPerChunk:(i+1)*numSamplesPerChunk]
            mean = np.mean(segment)
            x = [sample-mean for sample in segment] #dc filter
            # [b,a] = signal.butter(4, 15/(Fs/2), 'highpass')
            # y = signal.filtfilt(b,a,xdcf)
            #
            if( lsubstr(x, 0.01) ):
                print(fileBase + " ",  i)
                norm = librosa.util.normalize(x)
                sf.write("segmented" + "/" + fileBase + "--" + str(i) + ".wav", norm, sr)
            else:
                print("Discarding a silent segment ", i)
            i = i + 1;

max 151
SKR_1  0
max 122
SKR_1  1
max 154
SKR_1  2
max 143
SKR_1  3
max 132
SKR_1  4
max 150
SKR_1  5
max 153
SKR_1  6
max 151
SKR_1  7
max 127
SKR_1  8
max 160
SKR_1  9
max 128
SKR_1  10
max 148
SKR_1  11
max 143
SKR_1  12
max 117
SKR_1  13
max 145
SKR_1  14
max 114
SKR_1  15
max 133
SKR_1  16
max 157
SKR_1  17
max 132
SKR_1  18
max 141
SKR_1  19
max 131
SKR_1  20
max 127
SKR_1  21
max 161
SKR_1  22
max 153
SKR_1  23
max 144
SKR_1  24
max 167
SKR_1  25
max 142
SKR_1  26
max 149
SKR_1  27
max 154
SKR_1  28
max 138
SKR_1  29
max 136
SKR_1  30
max 155
SKR_1  31
max 142
SKR_1  32
max 159
SKR_1  33
max 114
SKR_1  34
max 102
SKR_1  35
max 142
SKR_1  36
max 108
SKR_1  37
max 110
SKR_1  38
max 119
SKR_1  39
max 107
SKR_1  40
max 116
SKR_1  41
max 90
SKR_1  42
max 121
SKR_1  43
max 132
SKR_1  44
max 132
SKR_1  45
max 122
SKR_1  46
max 123
SKR_1  47
max 117
SKR_1  48
max 104
SKR_1  49
max 125
SKR_1  50
max 134
SKR_1  51
max 135
SKR_1  52
max 160
SKR_1  53
max 135
SKR_1  54
max 123
SKR_1  55
max

KeyboardInterrupt: 

In [6]:
#Setup paths for paramManager
datapath = 'segmented'
parampath = 'segmentedParams'

In [7]:
pm=paramManager.paramManager(datapath,parampath) #filebase is directory name
pm.initParamFiles(overwrite=True) ##-----------   paramManager  interface ------------------##
pm.checkIntegrity()

paramList = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(parampath, os.path.splitext(f)[0] + ".params"))]
             
for p in paramList:
    soundText = os.path.splitext(p)[0].split("--")
    foo=pm.getParams(datapath + '/' + p) #extension is optional
    print(foo)
    pm.addMetaParam(parampath + '/' + p,"soundName", soundText[0])
    pm.addMetaParam(parampath + '/' + p,"segmentNum", soundText[1])

{'meta': {'filename': 'braids_REC_unedit--1113.wav'}}
{'meta': {'filename': 'testHaz-1--121.wav'}}
{'meta': {'filename': 'braids_REC_unedit--149.wav'}}
{'meta': {'filename': 'nether 96--277.wav'}}
{'meta': {'filename': 'em_Object_4--26.wav'}}
{'meta': {'filename': 'SKR_6--4.wav'}}
{'meta': {'filename': 'moog_wibbl_e--107.wav'}}
{'meta': {'filename': 'stochosV6freeS_rec 1--2.wav'}}
{'meta': {'filename': 'testHaz-2--199.wav'}}
{'meta': {'filename': 'em_Object_4--32.wav'}}
{'meta': {'filename': 'testHaz-1--135.wav'}}
{'meta': {'filename': 'briand_ex_3--19.wav'}}
{'meta': {'filename': 'briads_ex_1--4.wav'}}
{'meta': {'filename': 'nether 96--263.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1107.wav'}}
{'meta': {'filename': 'em_Object_2--88.wav'}}
{'meta': {'filename': 'briand_ex_3--31.wav'}}
{'meta': {'filename': 'braids_REC_unedit--175.wav'}}
{'meta': {'filename': 'netherGen_rec--246.wav'}}
{'meta': {'filename': 'braids_REC_unedit--613.wav'}}
{'meta': {'filename': 'nether_gen.rec--243.

{'meta': {'filename': 'SKR_1--30.wav'}}
{'meta': {'filename': 'netherGen_rec 1--219.wav'}}
{'meta': {'filename': 'arp_1--83.wav'}}
{'meta': {'filename': 'braids_REC_unedit--362.wav'}}
{'meta': {'filename': 'braids_REC_unedit--404.wav'}}
{'meta': {'filename': 'arp_10--23.wav'}}
{'meta': {'filename': 'arp_2--1.wav'}}
{'meta': {'filename': 'moog_sec_base_a--27.wav'}}
{'meta': {'filename': 'braids_REC_unedit--410.wav'}}
{'meta': {'filename': 'arp_10--37.wav'}}
{'meta': {'filename': 'testHaz-1--478.wav'}}
{'meta': {'filename': 'moog_sec_base_a--33.wav'}}
{'meta': {'filename': 'netherGen_rec 3--14.wav'}}
{'meta': {'filename': 'arp_1--97.wav'}}
{'meta': {'filename': 'braids_REC_unedit--376.wav'}}
{'meta': {'filename': 'SKR_1--24.wav'}}
{'meta': {'filename': 'testHaz-1--336.wav'}}
{'meta': {'filename': 'netherGen_rec 1--225.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1462.wav'}}
{'meta': {'filename': 'braids_REC_unedit--438.wav'}}
{'meta': {'filename': 'testHaz-1--450.wav'}}
{'meta': {'fi

{'meta': {'filename': 'nether_gen.rec--66.wav'}}
{'meta': {'filename': 'testHaz-2--206.wav'}}
{'meta': {'filename': 'em_Object_2--108.wav'}}
{'meta': {'filename': 'moog_sec_base_b--50.wav'}}
{'meta': {'filename': 'SKR_2--47.wav'}}
{'meta': {'filename': 'testHaz-1--296.wav'}}
{'meta': {'filename': 'braids_REC_unedit--76.wav'}}
{'meta': {'filename': 'moog_wibbl_e--53.wav'}}
{'meta': {'filename': 'em_Object_2--120.wav'}}
{'meta': {'filename': 'braids_REC_unedit--598.wav'}}
{'meta': {'filename': 'SKR_5--107.wav'}}
{'meta': {'filename': 'moog_wibbl_e--47.wav'}}
{'meta': {'filename': 'SKR_5--113.wav'}}
{'meta': {'filename': 'braids_REC_unedit--62.wav'}}
{'meta': {'filename': 'testHaz-1--282.wav'}}
{'meta': {'filename': 'braids_REC_unedit--201.wav'}}
{'meta': {'filename': 'arp_2--23.wav'}}
{'meta': {'filename': 'testHaz-1--269.wav'}}
{'meta': {'filename': 'braids_REC_unedit--89.wav'}}
{'meta': {'filename': 'arp_18--26.wav'}}
{'meta': {'filename': 'netherGen_rec--132.wav'}}
{'meta': {'filename

{'meta': {'filename': 'braids_REC_unedit--952.wav'}}
{'meta': {'filename': 'moog_sec_base_c--79.wav'}}
{'meta': {'filename': 'netherGen_rec 1--197.wav'}}
{'meta': {'filename': 'netherGen_rec 1--183.wav'}}
{'meta': {'filename': 'arp_12--69.wav'}}
{'meta': {'filename': 'nether_gen.rec--0.wav'}}
{'meta': {'filename': 'braids_REC_unedit--946.wav'}}
{'meta': {'filename': 'netherGen_rec 1--182.wav'}}
{'meta': {'filename': 'nether_gen.rec--1.wav'}}
{'meta': {'filename': 'braids_REC_unedit--947.wav'}}
{'meta': {'filename': 'arp_12--68.wav'}}
{'meta': {'filename': 'moog_sec_base_c--78.wav'}}
{'meta': {'filename': 'braids_REC_unedit--953.wav'}}
{'meta': {'filename': 'netherGen_rec 1--196.wav'}}
{'meta': {'filename': 'netherGen_rec 3--112.wav'}}
{'meta': {'filename': 'netherGen_rec 1--77.wav'}}
{'meta': {'filename': 'SKR_3--47.wav'}}
{'meta': {'filename': 'ichT-Xenak--93.wav'}}
{'meta': {'filename': 'moog_sec_base_c--50.wav'}}
{'meta': {'filename': 'moog_sec_base_c--137.wav'}}
{'meta': {'filename

{'meta': {'filename': 'nether_gen.rec--118.wav'}}
{'meta': {'filename': 'testHaz-1--520.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1512.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1274.wav'}}
{'meta': {'filename': 'nether 96--24.wav'}}
{'meta': {'filename': 'testHaz-1--246.wav'}}
{'meta': {'filename': 'nether 96--110.wav'}}
{'meta': {'filename': 'SKR_2--54.wav'}}
{'meta': {'filename': 'testHaz-2--215.wav'}}
{'meta': {'filename': 'nether_gen.rec--75.wav'}}
{'meta': {'filename': 'moog_sec_base_b--43.wav'}}
{'meta': {'filename': 'moog_wibbl_e--68.wav'}}
{'meta': {'filename': 'moog_sec_base_b--57.wav'}}
{'meta': {'filename': 'testHaz-2--201.wav'}}
{'meta': {'filename': 'nether_gen.rec--61.wav'}}
{'meta': {'filename': 'SKR_2--40.wav'}}
{'meta': {'filename': 'braids_REC_unedit--59.wav'}}
{'meta': {'filename': 'SKR_2--68.wav'}}
{'meta': {'filename': 'braids_REC_unedit--71.wav'}}
{'meta': {'filename': 'testHaz-1--291.wav'}}
{'meta': {'filename': 'SKR_5--100.wav'}}
{'meta': {'filenam

...

{'meta': {'filename': 'braids_REC_unedit--1278.wav'}}
{'meta': {'filename': 'nether 114--35.wav'}}
{'meta': {'filename': 'nether 96--108.wav'}}
{'meta': {'filename': 'braids_REC_unedit--236.wav'}}
{'meta': {'filename': 'arp_2--14.wav'}}
{'meta': {'filename': 'braids_REC_unedit--550.wav'}}
{'meta': {'filename': 'testHaz-1--538.wav'}}
{'meta': {'filename': 'nether_gen.rec--100.wav'}}
{'meta': {'filename': 'nether_gen.rec--86.wav'}}
{'meta': {'filename': 'netherGen_rec--105.wav'}}
{'meta': {'filename': 'arp_18--11.wav'}}
{'meta': {'filename': 'nether 114--7.wav'}}
{'meta': {'filename': 'SKR_5--124.wav'}}
{'meta': {'filename': 'moog_wibbl_e--70.wav'}}
{'meta': {'filename': 'em_Object_2--103.wav'}}
{'meta': {'filename': 'arp_10--7.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1287.wav'}}
{'meta': {'filename': 'braids_REC_unedit--55.wav'}}
{'meta': {'filename': 'SKR_2--58.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1293.wav'}}
{'meta': {'filename': 'braids_REC_unedit--41.wav'}}
{'met

{'meta': {'filename': 'netherGen_rec--258.wav'}}
{'meta': {'filename': 'netherGen_rec--92.wav'}}
{'meta': {'filename': 'em_Object_2--96.wav'}}
{'meta': {'filename': 'testHaz-1--103.wav'}}
{'meta': {'filename': 'nether 96--255.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1131.wav'}}
{'meta': {'filename': 'SKR_1--7.wav'}}
{'meta': {'filename': 'braids_REC_unedit--143.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1119.wav'}}
{'meta': {'filename': 'netherGen_rec--270.wav'}}
{'meta': {'filename': 'SKR_6--90.wav'}}
{'meta': {'filename': 'testHaz-2--193.wav'}}
{'meta': {'filename': 'braids_REC_unedit--625.wav'}}
{'meta': {'filename': 'stochosV6freeS_rec 1--8.wav'}}
{'meta': {'filename': 'nether_gen.rec--275.wav'}}
{'meta': {'filename': 'braids_REC_unedit--631.wav'}}
{'meta': {'filename': 'em_Object_4--38.wav'}}
{'meta': {'filename': 'nether_gen.rec--261.wav'}}
{'meta': {'filename': 'netherGen_rec--264.wav'}}
{'meta': {'filename': 'SKR_6--84.wav'}}
{'meta': {'filename': 'testHaz-2--187.

{'meta': {'filename': 'braids_REC_unedit--354.wav'}}
{'meta': {'filename': 'braids_REC_unedit--432.wav'}}
{'meta': {'filename': 'arp_10--15.wav'}}
{'meta': {'filename': 'moog_sec_base_a--11.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1468.wav'}}
{'meta': {'filename': 'SKR_7--94.wav'}}
{'meta': {'filename': 'SKR_7--80.wav'}}
{'meta': {'filename': 'braids_REC_unedit--426.wav'}}
{'meta': {'filename': 'testHaz-1--328.wav'}}
{'meta': {'filename': 'netherGen_rec 3--22.wav'}}
{'meta': {'filename': 'braids_REC_unedit--340.wav'}}
{'meta': {'filename': 'SKR_1--12.wav'}}
{'meta': {'filename': 'netherGen_rec 1--213.wav'}}
{'meta': {'filename': 'testHaz-1--300.wav'}}
{'meta': {'filename': 'arp_1--89.wav'}}
{'meta': {'filename': 'braids_REC_unedit--368.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1332.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1454.wav'}}
{'meta': {'filename': 'arp_10--29.wav'}}
{'meta': {'filename': 'testHaz-1--466.wav'}}
{'meta': {'filename': 'testHaz-1--472.wav'}}


{'meta': {'filename': 'arp_10--110.wav'}}
{'meta': {'filename': 'SKR_1--101.wav'}}
{'meta': {'filename': 'ichT-Xenak--59.wav'}}
{'meta': {'filename': 'arp_10--104.wav'}}
{'meta': {'filename': 'braids_REC_unedit--769.wav'}}
{'meta': {'filename': 'moog_sec_base_a--151.wav'}}
{'meta': {'filename': 'testHaz-2--68.wav'}}
{'meta': {'filename': 'netherGen_rec 1--174.wav'}}
{'meta': {'filename': 'em_Object_1--31.wav'}}
{'meta': {'filename': 'moog_wibbl_b--23.wav'}}
{'meta': {'filename': 'nether 96--331.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1055.wav'}}
{'meta': {'filename': 'em_Object_2--7.wav'}}
{'meta': {'filename': 'em_Object_1--19.wav'}}
{'meta': {'filename': 'netherGen_rec 1--95.wav'}}
{'meta': {'filename': 'nether 96--319.wav'}}
{'meta': {'filename': 'arp_3--16.wav'}}
{'meta': {'filename': 'ichT-Xenak--71.wav'}}
{'meta': {'filename': 'SKR_5--37.wav'}}
{'meta': {'filename': 'braids_REC_unedit--741.wav'}}
{'meta': {'filename': 'testHaz-2--40.wav'}}
{'meta': {'filename': 'braids_R

{'meta': {'filename': 'braids_REC_unedit--78.wav'}}
{'meta': {'filename': 'SKR_2--49.wav'}}
{'meta': {'filename': 'arp_10--2.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1282.wav'}}
{'meta': {'filename': 'braids_REC_unedit--50.wav'}}
{'meta': {'filename': 'testHaz-2--208.wav'}}
{'meta': {'filename': 'nether 114--2.wav'}}
{'meta': {'filename': 'nether_gen.rec--68.wav'}}
{'meta': {'filename': 'SKR_5--121.wav'}}
{'meta': {'filename': 'moog_wibbl_e--75.wav'}}
{'meta': {'filename': 'em_Object_2--106.wav'}}
{'meta': {'filename': 'moog_wibbl_e--61.wav'}}
{'meta': {'filename': 'em_Object_2--112.wav'}}
{'meta': {'filename': 'braids_REC_unedit--1296.wav'}}
{'meta': {'filename': 'braids_REC_unedit--44.wav'}}
{'meta': {'filename': 'testHaz-2--42.wav'}}
{'meta': {'filename': 'braids_REC_unedit--743.wav'}}
{'meta': {'filename': 'SKR_5--35.wav'}}
{'meta': {'filename': 'ichT-Xenak--73.wav'}}
{'meta': {'filename': 'arp_3--14.wav'}}
{'meta': {'filename': 'netherGen_rec 1--97.wav'}}
{'meta': {'filena

In [5]:
paramList = ["arp_1--3.params"]

for p in paramList:
    datapath = os.path.splitext(p)[0].split("--")[0]
    foo=pm.getParams(parampath + '/' + p) #extension is optional
    print(foo)

{'meta': {'filename': 'arp_1--3.wav', 'soundName': 'arp_1', 'segmentNum': '3'}}


In [9]:
#Generator SGJSOn
import json

SGjson = {}
SGjson["records"] = []
sgrecord = {}

paramList = [f for f in os.listdir(parampath)]

for filename in paramList:
        json_file = open(parampath + "/" + filename);
        data = json.load(json_file)
        sgrecord = {}
        sgrecord["sound_name"] = data["meta"]["soundName"] 
        sgrecord["samplerate"] = 22050; 
        sgrecord["sound_source"] = 1; #acoustic
        sgrecord["segment_number"] = data["meta"]["segmentNum"]
        sgrecord["sound_string"] = filename; #unique string with parameters
        SGjson[filename]=sgrecord
        #SGjson["records"].append(sgrecord)
    
print(json.dumps(SGjson, indent=4, sort_keys=True))

{
    "Reaktor 5_rec 4--0.params": {
        "samplerate": 22050,
        "segment_number": "0",
        "sound_name": "Reaktor 5_rec 4",
        "sound_source": 1,
        "sound_string": "Reaktor 5_rec 4--0.params"
    },
    "Reaktor 5_rec 4--1.params": {
        "samplerate": 22050,
        "segment_number": "1",
        "sound_name": "Reaktor 5_rec 4",
        "sound_source": 1,
        "sound_string": "Reaktor 5_rec 4--1.params"
    },
    "Reaktor 5_rec 4--10.params": {
        "samplerate": 22050,
        "segment_number": "10",
        "sound_name": "Reaktor 5_rec 4",
        "sound_source": 1,
        "sound_string": "Reaktor 5_rec 4--10.params"
    },
    "Reaktor 5_rec 4--11.params": {
        "samplerate": 22050,
        "segment_number": "11",
        "sound_name": "Reaktor 5_rec 4",
        "sound_source": 1,
        "sound_string": "Reaktor 5_rec 4--11.params"
    },
    "Reaktor 5_rec 4--12.params": {
        "samplerate": 22050,
        "segment_number": "12",
       

### Stores as single SG json record

In [10]:

with open('SGrecords.json', 'w') as outfile:
    json.dump(SGjson, outfile, indent=4, sort_keys=True)